# Importations

In [1]:
import os
import sys

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
from tqdm import tqdm
import neuroseries as nts
import scipy as sp
import scipy.stats
import seaborn as sns

import bk.load
import bk.compute
import bk.plot
import bk.signal
import mc.functions

import matplotlib as mpl
mpl.rcParams["svg.fonttype"] = "none"

%matplotlib qt

# Variables

In [118]:
bk.load.current_session_linux(base_folder= 'E:/DATA/GG-Dataset/',local_path= 'Rat11/Rat11-20150323')
#load the path name, session name, rat number, day number, and the number of channels in the session 

Rat : 11 on day : 12
Working with session Rat11-20150323 @ E:/DATA/GG-Dataset/Rat11/Rat11-20150323


True

In [119]:
states = bk.load.states() 
#load the time intervals of the states of the animal (Rem, sws, wake, drowsy) in a dict

In [120]:
neurons,metadata = bk.load.spikes() 
#load the metadata (pd dataframe where informations about the neurons are: 
#rat, day, shank, id on the shank, region, and type)

#load the neurons (np array containing nt frames from each neuron, 
#in each nt frame there is the timing of the spike (index) and values are nan)

Data already saved in Numpy format, loading them from here:
Rat11-20150323-neurons.npy
Rat11-20150323-metadata.npy


In [121]:
neurons_pyr = neurons[metadata['Type'] == 'Pyr'] #only pyramidal neurons
neurons_pyr_hpc = neurons[(metadata['Type'] == 'Pyr') & (metadata['Region'] == 'Hpc')] #only hippocampal pyramidal 
neurons_pyr_bla = neurons[(metadata['Type'] == 'Pyr') & (metadata['Region'] == 'BLA')] #only bla pyramidal

neurons_int = neurons[metadata['Type'] == 'Int'] #only interneurons
neurons_int_hpc = neurons[(metadata['Type'] == 'Int') & (metadata['Region'] == 'Hpc')] #only hippocampal interneurons 
neurons_int_bla = neurons[(metadata['Type'] == 'Int') & (metadata['Region'] == 'BLA')] #only bla interneurons

neurons_pyr_ordered = np.concatenate((neurons_pyr_hpc, neurons_pyr_bla)) #all pyramidal neurons, but in order
neurons_int_ordered = np.concatenate((neurons_int_hpc, neurons_int_bla)) #all interneurons, but in order

In [122]:
if len(neurons_pyr_bla) == 0 or len(neurons_int_bla) == 0:
    print('Careful, there are not enough neurons')

In [123]:
window = 1 #the size of the time bins used to make the correlation matrix, in sec
smallbins = 0.1 #the size of the small bins used to compute binspikes
shift = 1 #the shift from one bin to another, in sec 

# Functions 

In [2]:
def mean_synchrony(neurons, asymetry=False, neurons2=False, window=1, smallbins=0.1, shift=1):
    '''
    MC 29/10/21
    This function computes the mean synchrony for neurons across time
    Inputs: 
        neurons: the neurons you want to compute the synchrony on
        smallbins: the size of the time bins used to compute the number of spikes per neuron in that bin (in sec), default = 0.1
        window: the size of the time window in which to compute the mean of synchrony among the neurons (in sec), default = 1
        shift: the shift between one window and the following (in sec), default = 1
        asymetry: if you want to compute this synchrony between 2 different populations of neurons, default = False
        neurons2: the second population of neurons, default = False
    Outputs:
        a neuroseries data frame with the mean of synchrony across time
    '''
    if asymetry: #if we want to compute the synchrony between 2 different populations of neurons
        neurons = np.concatenate((neurons, neurons2)) # concatenate the 2 populations of neurons
        l_neurons2 = len(neurons2) #take the length of the second population, it will be used to extract
                                   #only the part of the corrcoef matrix between the 2 populations and not between
                                   #themselves
    t,binned_neurons = bk.compute.binSpikes(neurons, binSize=smallbins) #count the number of spikes for each neuron in smallbins
    b = int(window/smallbins) #the number of small bins in one window
    c = int(shift/smallbins) #the number of small bins you have to shift each time
    mean_sync = np.zeros(int(binned_neurons.shape[1]/c)) #initialisation of the mean synchrony across time 
    start = int(0 + b/2) #where to start the computation of mean synchrony (you have to start at b because you
                       #compute corrcoef in [-b/2,b/2] windows)
    stop = int(binned_neurons.shape[1]-b/2) #where to end the computation of mean synchrony (at the end of the recording)
    for i,j in enumerate(tqdm(range(start, stop, c))): #for each window
        corrcoef = np.corrcoef(binned_neurons[:,int(j-b/2):int(j + b/2)]) #compute the corrcoef in a [-b/2:b/2] window
        if asymetry: 
            corrcoef = corrcoef[l_neurons2:,:l_neurons2] #extract only the corrcoef matrix between the 2 populations
        mean_sync[i] = np.nanmean(corrcoef) #compute the mean of corrcoef for each window, disregarding the nan
    t=t[::c] #extract t timings at c steps
    if len(t) == len(mean_sync):
        toreturn = nts.Tsd(t, mean_sync, time_units='s')
    else: 
        toreturn = nts.Tsd(t[:-1], mean_sync, time_units='s') #create a neuroseries frame with the means of synchrony
    return toreturn

In [3]:
def mean_synchrony_nan(neurons, asymetry=False, neurons2=False, window=2, smallbins=0.1, shift=1):
    '''
    MC 02/11/21
    This function computes the mean synchrony for neurons across time, but with NaN values changed to 0
    Inputs: 
        neurons: the neurons you want to compute the synchrony on
        smallbins: the size of the time bins used to compute the number of spikes per neuron in that bin (in sec), default = 0.1
        window: the size of the time window in which to compute the mean of synchrony among the neurons (in sec), default = 2
        shift: the shift between one window and the following (in sec), default = 1
        asymetry: if you want to compute this synchrony between 2 different populations of neurons, default = False
        neurons2: the second population of neurons, default = False
    Outputs:
        a neuroseries data frame with the mean of synchrony across time
    '''
    if asymetry: #if we want to compute the synchrony between 2 different populations of neurons
        neurons = np.concatenate((neurons, neurons2)) # concatenate the 2 populations of neurons
        l_neurons2 = len(neurons2) #take the length of the second population, it will be used to extract
                                   #only the part of the corrcoef matrix between the 2 populations and not between
                                   #themselves
    t,binned_neurons = bk.compute.binSpikes(neurons, binSize=smallbins) #count the number of spikes for each neuron in smallbins
    b = int(window/smallbins) #the number of small bins in one window
    c = int(shift/smallbins) #the number of small bins you have to shift each time
    mean_sync = np.zeros(int(binned_neurons.shape[1]/c)) #initialisation of the mean synchrony across time 
    start = int(0 + b/2) #where to start the computation of mean synchrony (you have to start at b because you
                       #compute corrcoef in [-b/2,b/2] windows)
    stop = int(binned_neurons.shape[1]-b/2) #where to end the computation of mean synchrony (at the end of the recording)
    for i,j in enumerate(tqdm(range(start, stop, c))): #for each window
        corrcoef = np.corrcoef(binned_neurons[:,int(j-b/2):int(j + b/2)]) #compute the corrcoef in a [-b/2:b/2] window
        if asymetry: 
            corrcoef = corrcoef[l_neurons2:,:l_neurons2] #extract only the corrcoef matrix between the 2 populations
        corrcoef[np.isnan(corrcoef)] = 0 #all nan values are changed to 0
        mean_sync[i] = np.nanmean(corrcoef) #compute the mean of corrcoef for each window, disregarding the nan
    t=t[::c] #extract t timings at c steps
    toreturn = nts.Tsd(t[:-1], mean_sync, time_units='s') #create a neuroseries frame with the means of synchrony
    return toreturn

In [4]:
def mean_firing_rate(neurons, window=2, smallbins=0.1, shift=1):
    '''
    MC 02/11/21
    This function computes the mean firing rate of a neuron population across time
    Inputs: 
        neurons: the neurons you want to compute the mean firing rate on
        smallbins: the size of the time bins used to compute the number of spikes per neuron in that bin (in sec), default = 0.1
        window: the size of the time window in which to compute the mean of firing rate among the neurons (in sec), default = 2
        shift: the shift between one window and the following (in sec), default = 1
    Outputs:
        a neuroseries data frame with the mean of firing rate across time
    '''
    t,binned_neurons = bk.compute.binSpikes(neurons, binSize=smallbins) #count the number of spikes for each neuron in smallbins
    b = int(window/smallbins) #the number of small bins in one window
    c = int(shift/smallbins) #the number of small bins you have to shift each time
    mean_fr = np.zeros(int(binned_neurons.shape[1]/c)) #initialisation of the mean firing rate across time 
    start = int(0 + b/2) #where to start the computation of mean synchrony (you have to start at b because you
                       #compute corrcoef in [-b/2,b/2] windows)
    stop = int(binned_neurons.shape[1]-b/2) #where to end the computation of mean synchrony (at the end of the recording)
    for i,j in enumerate(tqdm(range(start, stop, c))): #for each window
        mean_fr[i] = np.nanmean(binned_neurons[:,int(j-b/2):int(j + b/2)])/window #compute the mean of firing rate for each window, 
                                                                                  #disregarding the nan
    t=t[::c] #extract t timings at c steps
    if len(t) == len(mean_fr):
        toreturn = nts.Tsd(t, mean_fr, time_units='s')
    else: 
        toreturn = nts.Tsd(t[:-1], mean_fr, time_units='s') #create a neuroseries frame with the means of firing rate
    return toreturn

In [5]:
def pourcent_active_neurons(neurons, window=1):
    '''
    MC 03/11/21
    This function computes the pourcentage of active neurons across time
    Inputs: 
        neurons: the neurons you want to compute the pourcentages on
        window: the size of the time window in which to compute the pourcentage of active neurons (in sec), default = 1
    Outputs:
        a neuroseries data frame with the pourcentage of active neurons across time
    '''
    t,binned_neurons = bk.compute.binSpikes(neurons, binSize=window) #count the number of spikes for each neuron in windows
    pourcent_active = np.zeros(int(binned_neurons.shape[1])) #initialisation of the pourcentages across time 
    for i,j in enumerate(tqdm(range(0, int(binned_neurons.shape[1]), window))): #for each window
        pourcent_active[i] = np.count_nonzero(binned_neurons[:,j])/len(neurons) #compute the pourcentage for each window
    t=t[::window] #extract t timings at shift steps
    toreturn = nts.Tsd(t, pourcent_active, time_units='s') #create a neuroseries frame with the pourcentages
    return toreturn

In [6]:
def normalization(neurons):
    '''
    MC 08/11/21
    this function normalizes the pourcentage of active cells to the mean firing rate
    Input : the neurons you want to compute this normalization on
    Output : a neuroseries timeframe of this normalized pourcentages of active cells 
    '''
    t,_ = bk.compute.binSpikes(neurons, binSize=1)
    pac = pourcent_active_neurons(neurons, window = 1) #compute the pourcent of active cells across time
    pac = bk.compute.nts_smooth(pac, 100, 50) #smoothing the results 
    mfr = mean_firing_rate(neurons, window=2, shift=1) #compute the mean FR across time 
    mfr = bk.compute.nts_smooth(mfr, 100, 50) #smoothing the results
    toreturn = [(i/j) for i,j in zip(pac.values, mfr.values)] #normalize the pac to the mfr
    toreturn = nts.Tsd(t, toreturn, time_units = 's') #create the tsd
    return toreturn
    

In [7]:
def synchrony_around_transitions(transitions, mean_sync):
    '''
    MC 13/12/21
    This function computes the mean of synchrony around transitions REM/sws or sws/REM, 
    and the mean of synchrony of the whole epochs considered (REM or sws)
    Inputs: 
        transitions: the timing of the transitions
        mean_sync: the mean synchrony across the session
    Output: a dataframe with the different means for each type 
    '''
    
    trans = {'Rem before transition': [], 
            'Rem after transition': [],
             'Rem': [],
            'sws before transition' : [], 
            'sws after transition': [],
            'sws': []}
    
    transitions_rem_sws = transitions[1][('Rem', 'sws')].index.values
    transitions_sws_rem = transitions[1][('sws', 'Rem')].index.values
    
    for i  in transitions_rem_sws:
        interval_rem = nts.IntervalSet(i-10_000_000, i, time_units='us')
        trans['Rem before transition'].append(np.mean(mean_sync.restrict(interval_rem).values))
        interval_sws = nts.IntervalSet(i, i+10_000_000, time_units='us')
        trans['sws after transition'].append(np.mean(mean_sync.restrict(interval_sws).values))

    for i  in transitions_sws_rem:
        interval_sws = nts.IntervalSet(i-10_000_000, i, time_units='us')
        trans['sws before transition'].append(np.mean(mean_sync.restrict(interval_sws).values))
        interval_rem = nts.IntervalSet(i, i+10_000_000, time_units='us')
        trans['Rem after transition'].append(np.mean(mean_sync.restrict(interval_rem).values))

    for i, j  in zip(states['Rem'].as_units('s').start, states['Rem'].as_units('s').end):
        interval = nts.IntervalSet(i,j, time_units='s')
        trans['Rem'].append(np.mean(mean_sync.restrict(interval).values))

    for i, j  in zip(states['sws'].as_units('s').start, states['sws'].as_units('s').end):
        interval = nts.IntervalSet(i,j, time_units='s')
        trans['sws'].append(np.mean(mean_sync.restrict(interval).values))

    trans = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in trans.items() ]))
    return trans

In [8]:
def plot_synchrony_over_time_transitions(transitions, mean_sync, rem_to_sws=True):
    '''
    MC 15/12/21
    This function draws the synchrony over time around the transitions
    TO CONTINUE
    Inputs:
        transitions: the timing of the transitions 
        mean_sync: the mean of synchrony over time 
        rem_to_sws: if we look at rem to sws transitions or sws to rem transitions (by default: rem_to_sws)
    Output:
    A figure with the synchrony over time drawn
    '''
    means =[]
    fig, ax = plt.subplots()
    for i  in transitions_rem_sws:
        interval = nts.IntervalSet(i-20_000_000, i+ 40_000_000, time_units='us')
        m = mean_sync_pyr_bla.restrict(interval).values
        means.append(m)
        plt.plot(m, color = 'grey', alpha = 0.3)
    a = mpatches.Rectangle((0,0), 20, 0.3, facecolor = 'orange', alpha = 0.5)
    ax.add_patch(a)
    mean = np.mean(means, axis=0)
    plt.plot(mean, color = 'r', linewidth=4)

In [9]:
def corr_coeff_without_nan(var1, var2):
    '''
    MC 25/12/21
    This function returns 
    Inputs:
        var1: the first variable we want to extract the non-NaN from
        var2: the second variable we want to extract the non-NaN from
    Outputs:
        var1: the first variable masked
        var2: the second variable masked
        mask: the mask (the position where there are non-NaN in the variables)
    
    '''
    
    mask = ~np.logical_or(np.isnan(var1), np.isnan(var2)) 
    #create a mask (a boolean array) that tells if there are NaN in 
    #either variable for each position

    var1 = np.compress(mask, var1)
    var2 = np.compress(mask, var2)
    #extract only the positions where there are non NaN in either variable
    
    return var1, var2, mask

In [10]:
def get_first_and_last_intervals(states):
    '''
    MC 29/12/2021
    This function returns the first and the last epochs of a given state
    
    Inputs: 
        states: the given state you want to first and last epochs from
    Outputs: 
        start: the first epoch of the given state
        last: the last epoch of the given state
    '''
    first = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in states.iloc[0].items() ])) 
    #take the first line of the given state, and return a dataframe of it 
    first = nts.IntervalSet(first) 
    #turn it into a neurotimeseries interval (to be compatible with .restrict)
    last = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in states.iloc[-1].items() ]))
    last = nts.IntervalSet(last)
    return first, last

In [11]:
def get_extended_sleep(states,sleep_th,wake_th):
    '''
    BK 03/01/22
    Return extended sleep session given state, sleep
    Inputs:
        states: the given states you want the extended sleep from
        sleep_th: the threshold of sleep
        wake_th: the threshold of wake
    Outputs:
        ext_sleep: the epochs of extended sleep 
    '''
    
    ext_sleep = states['sws'].union(states['Rem']).merge_close_intervals(wake_th,time_units = 's')
    ext_sleep = ext_sleep[ext_sleep.duration(time_units='s')>sleep_th].reset_index(drop = True)
    return ext_sleep

In [12]:
def iterate_over_sessions(function): 
    '''
    MC 09/01/22
    Inputs:
        function: the function you want to apply to all sessions considered
    Outputs: 
        output_dict: a dictionary with the sessions as keys and the results of the function as values 
    '''
    os.chdir('E:/DATA/GG-Dataset/')
    session_index = pd.read_csv("relative_session_indexing.csv")
    
    to_ignore = ["Rat08/Rat08-20130720", 
             "Rat08-20130722", 
             "Rat08-20130718",
             "Rat09-20140408",
             "Rat09-20140409",
             "Rat10-20140619",
             "Rat10-20140620",
             "Rat10-20140622",
             "Rat10-20140708",
             "Rat11-20150309",
             "Rat11-20150318",
             "Rat11-20150319", 
             "Rat11-20150324",
             "Rat11-20150327",
             "Rat11-20150401"]
    
    output_dict = {}
    
    for path in tqdm(session_index["Path"]):
        
        if path in to_ignore:
            print(f'{path} THIS SESSION IS BULLSHIT')
            continue
        try:
            output = function(path)
            output_dict.update({bk.load.session : output})
        except:
            print(f'{path} ERROR IN THIS SESSION, TO WORK IT THROUGH')
            continue
        print(output)
    return output_dict

# Computation of variables

In [16]:
mean_sync_pyr_bla = mean_synchrony(neurons_pyr_bla, window=1, smallbins=0.1, shift=1)
mean_sync_int_bla = mean_synchrony(neurons_int_bla, window=1, smallbins=0.1, shift=1)

mean_sync_pyr_bla_s = bk.compute.nts_smooth(mean_sync_pyr_bla, 100,50)
mean_sync_int_bla_s = bk.compute.nts_smooth(mean_sync_int_bla, 100,50)

  0%|                                                                                        | 0/24290 [00:00<?, ?it/s]C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
 20%|██████████████▋                                                            | 4744/24290 [00:02<00:07, 2560.21it/s]C:\Users\maell\AppData\Local\Temp/ipykernel_20972/1913985095.py:31: RuntimeWarning: Mean of empty slice
  mean_sync[i] = np.nanmean(corrcoef) #compute the mean of corrcoef for each window, disregarding the nan
100%|██████████████████████████████████████████████████████████████████████████| 24290/24290 [00:09<00:00, 2468.97it/s]


In [67]:
mean_sync_nan_pyr_bla = mean_synchrony_nan(neurons_pyr_bla, window=1, smallbins=0.1, shift=1)
mean_sync_nan_int_bla = mean_synchrony_nan(neurons_int_bla, window=1, smallbins=0.1, shift=1)

mean_sync_nan_pyr_bla_s = bk.compute.nts_smooth(mean_sync_nan_pyr_bla, 100,50)
mean_sync_nan_int_bla_s = bk.compute.nts_smooth(mean_sync_nan_int_bla, 100,50)

100%|██████████████████████████████████████████████████████████████████████████| 25510/25510 [00:09<00:00, 2572.78it/s]


In [68]:
m_asym_bla = mean_synchrony(neurons_pyr_bla, asymetry=True, 
                            neurons2=neurons_int_bla, window=1, 
                            smallbins = 0.1, shift= 1)

m_asym_bla_s = bk.compute.nts_smooth(m_asym_bla, 30, 30)

  0%|                                                                                        | 0/25510 [00:00<?, ?it/s]C:\Users\maell\AppData\Local\Temp/ipykernel_25084/453570434.py:31: RuntimeWarning: Mean of empty slice
  mean_sync[i] = np.nanmean(corrcoef) #compute the mean of corrcoef for each window, disregarding the nan
100%|███████████████████████████████████████████████████████████████████████████| 25510/25510 [00:26<00:00, 974.15it/s]


In [17]:
mean_fr_pyr_bla = mean_firing_rate(neurons_pyr_bla)
mean_fr_int_bla = mean_firing_rate(neurons_int_bla)

mean_fr_pyr_bla_s = bk.compute.nts_smooth(mean_fr_pyr_bla, 100,50)
mean_fr_int_bla_s = bk.compute.nts_smooth(mean_fr_int_bla, 100,50)

100%|█████████████████████████████████████████████████████████████████████████| 24289/24289 [00:00<00:00, 26842.45it/s]


In [70]:
pourcent_active_pyr_bla = pourcent_active_neurons(neurons_pyr_bla)
pourcent_active_int_bla = pourcent_active_neurons(neurons_int_bla)

pourcent_active_pyr_bla_s = bk.compute.nts_smooth(pourcent_active_pyr_bla, 100,50)
pourcent_active_int_bla_s = bk.compute.nts_smooth(pourcent_active_int_bla, 100,50)

100%|████████████████████████████████████████████████████████████████████████| 25510/25510 [00:00<00:00, 176399.48it/s]


In [71]:
transitions = bk.compute.transitions_times(states)

transitions_rem_sws = transitions[1][('Rem', 'sws')].index.values
transitions_sws_rem = transitions[1][('sws', 'Rem')].index.values

# Matrices de corrélation et calculs de moyenne de synchronie

### Asymetrical matrices BLA/HPC with pyr and int

In [45]:
m_asym_pyr = mean_synchrony(neurons_pyr_bla, asymetry=True, neurons2=neurons_pyr_hpc, window=1, smallbins = 0.025, shift= 1)
m_asym_int = mean_synchrony(neurons_int_bla, asymetry=True, neurons2=neurons_int_hpc, window=1, smallbins = 0.025, shift= 1)
m_asym_bla = mean_synchrony(neurons_pyr_bla, asymetry=True, neurons2=neurons_int_bla, window=1, smallbins = 0.025, shift= 1)
m_asym_hpc = mean_synchrony(neurons_pyr_hpc, asymetry=True, neurons2=neurons_int_hpc, window=1, smallbins = 0.025, shift= 1)

  0%|                                                                                        | 0/22639 [00:00<?, ?it/s]C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
  4%|██▋                                                                          | 808/22639 [00:00<00:22, 987.60it/s]C:\Users\maell\AppData\Local\Temp/ipykernel_23364/4247394187.py:27: RuntimeWarning: Mean of empty slice
  mean_sync[i] = np.nanmean(corrcoef)
100%|██████████████████████████████████████████████████████████████████████████| 22639/22639 [00:19<00:00, 1162.21it/s]


In [46]:
m_asym_pyr_s = bk.compute.nts_smooth(m_asym_pyr, 10,5)
m_asym_int_s = bk.compute.nts_smooth(m_asym_int, 10,5)
m_asym_bla_s = bk.compute.nts_smooth(m_asym_bla, 10,5)
m_asym_hpc_s = bk.compute.nts_smooth(m_asym_hpc, 10,5)

In [65]:
#for asymetrical matrices (between hpc and bla)
#To get the correlation coefficient between hpc and bla, I concatenate the 2, then I compute the matrix 
#but I only take the part where it is between the two, and not between themselves. Then I can compute the mean

l_pyr_bla = len(neurons_pyr_bla)
l_pyr_hpc = len(neurons_pyr_hpc)
l_int_bla = len(neurons_int_bla)
l_int_hpc = len(neurons_int_hpc)
#compute their length to only take the part of the matrix where it is asymetrical

mean_corr_pyr = []
mean_corr_int = []

for i in interval:
    corrcoef_pyr = np.corrcoef(binned_pyr[:,int(i):int(i + bins)]) 
    corrcoef_pyr = corrcoef_pyr[l_pyr_hpc:-1, l_pyr_bla:-1]
    mean_corr_pyr.append(np.nanmean(corrcoef_pyr))
    corrcoef_int = np.corrcoef(binned_int[:,int(i):int(i + bins)])  
    corrcoef_int = corrcoef_int[l_int_hpc:-1, l_int_bla:-1]
    mean_corr_int.append(np.nanmean(corrcoef_int))


C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


### Plotting

In [48]:
plt.figure()
plt.subplot(4,1,1)
bk.plot.intervals(states['Rem'])
bk.plot.intervals(states['sws'], col='grey')
plt.plot(m_pyr_bla[15000:].as_units('s'), label='pyr bla', color = 'g')
plt.plot(m_pyr_hpc[15000:].as_units('s'), label='pyr hpc', color = 'b')
plt.plot(m_asym_pyr[15000:].as_units('s'), label ='synchro pyr BLA/HPC', color='r')
plt.xlim(15000,22000)
plt.legend()

plt.subplot(4,1,2)
bk.plot.intervals(states['Rem'])
bk.plot.intervals(states['sws'], col='grey')
plt.plot(m_int_bla[15000:].as_units('s'), label='int bla', color = 'g', linestyle='--')
plt.plot(m_int_hpc[15000:].as_units('s'), label='int hpc', color = 'b', linestyle='--')
plt.plot(m_asym_int[15000:].as_units('s'), label ='synchro int BLA/HPC', color='r')
plt.xlim(15000,22000)
plt.legend()

plt.subplot(4,1,3)
bk.plot.intervals(states['Rem'])
bk.plot.intervals(states['sws'], col='grey')
plt.plot(m_pyr_bla[15000:].as_units('s'), label='pyr bla', color = 'g')
plt.plot(m_int_bla[15000:].as_units('s'), label='int bla', color = 'g', linestyle='--')
plt.plot(m_asym_bla[15000:].as_units('s'), label ='synchro BLA pyr/int', color='r')
plt.xlim(15000,22000)
plt.legend()

plt.subplot(4,1,4)
bk.plot.intervals(states['Rem'])
bk.plot.intervals(states['sws'], col='grey')
plt.plot(m_pyr_hpc[15000:].as_units('s'), label='pyr hpc', color = 'b')
plt.plot(m_int_hpc[15000:].as_units('s'), label='int hpc', color = 'b', linestyle='--')
plt.plot(m_asym_hpc[15000:].as_units('s'), label ='synchro HPC Pyr/int', color='r')
plt.xlim(15000,22000)
plt.legend()

In [23]:
bk.plot.intervals(states['Rem'])
bk.plot.intervals(states['sws'], col='grey')
plt.plot(mean_fr_pyr_bla_s.as_units('s'), label='mean firing rate', color = 'g')
plt.plot(pourcent_active_pyr_bla_s.as_units('s'), label='pourcent active', color = 'b')
plt.plot(mean_sync_pyr_bla_s.as_units('s'), label='mean synchrony', color = 'r')

plt.legend()

In [ ]:
plt.figure()
plt.subplot(1,2,1)
g = sns.regplot(mean_fr_pyr_bla_s.restrict(states['Rem']).values, 
                mean_sync_pyr_bla_s.restrict(states['Rem']).values, 
                scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, x_ci='sd', color='orange')
sns.regplot(mean_fr_pyr_bla_s.restrict(states['sws']).values, 
            mean_sync_pyr_bla_s.restrict(states['sws']).values, 
            scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, x_ci='sd', color='grey')
sns.regplot(mean_fr_pyr_bla_s[100:-100].restrict(states['wake']).values, 
            mean_sync_pyr_bla_s[100:-100].restrict(states['wake']).values, 
            scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, x_ci='sd')
g.set(xlabel ='mean firing rate', ylabel = 'mean synchrony',
      title='Correlation between synchrony and firing rate in BLA pyramidal cells')
g.legend(labels=['Rem', 'sws', 'Wake'])

plt.subplot(1,2,2)
g = sns.regplot(mean_fr_int_bla_s.restrict(states['Rem']).values, 
                mean_sync_nan_int_bla_s.restrict(states['Rem']).values, 
                scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, robust=True, x_ci='sd', color='orange')
sns.regplot(mean_fr_int_bla_s.restrict(states['sws']).values, 
            mean_sync_nan_int_bla_s.restrict(states['sws']).values, 
            scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, robust=True, x_ci='sd', color='grey')
sns.regplot(mean_fr_int_bla_s[100:-100].restrict(states['wake']).values, 
            mean_sync_nan_int_bla_s[100:-100].restrict(states['wake']).values, 
            scatter_kws={'alpha':0.1, 's':5}, robust=True, line_kws={'lw':5}, x_ci='sd')
g.set(xlabel ='mean firing rate', ylabel = 'pourcent of active cells',
      title='Correlation between pourcent of active cells and firing rate in BLA pyramidal cells')
g.legend(labels=['Rem', 'sws', 'Wake'])

In [88]:
plt.subplot(1,2,1)
g = sns.regplot(mean_fr_pyr_bla_s[100:-100].values, mean_sync_pyr_bla_s[100:-100].values, 
                scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, x_ci='sd', color='grey')
g.set(xlabel ='mean firing rate', ylabel = 'mean synchrony',
      title='Correlation between synchrony and firing rate in BLA pyramidal')

plt.subplot(1,2,2)
g = sns.regplot(mean_fr_pyr_bla_s[100:-100].values, pourcent_active_pyr_bla_s[100:-100].values, 
                scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, x_ci='sd', color='grey')
g.set(xlabel ='mean firing rate', ylabel = 'mean pourcent active',
      title='Correlation between pourcent of active cells and firing rate in BLA pyramidal')

NameError: name 'mean_fr_pyr_bla_s' is not defined

In [31]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(pourcent_active_pyr_bla_s[100:-100].values,
           mean_sync_pyr_bla_s[100:-100].values,
           mean_fr_pyr_bla_s[100:-100].values)

In [34]:
#plot des moyennes de synchronie en fonction des states 
plt.subplot(2,1,1)
bk.plot.intervals(states['Rem'])
bk.plot.intervals(states['sws'], col='grey')
plt.plot(mean_pyr_bla_s.as_units('s'), color='g',linewidth=2, label='pyr BLA')
plt.plot(mean_pyr_hpc_s.as_units('s'), color='b',linewidth=2, label='pyr HPC')
plt.legend()

plt.subplot(2,1,2)
bk.plot.intervals(states['Rem'])
bk.plot.intervals(states['sws'], col='grey')
plt.plot(mean_int_bla_s.as_units('s'), color='g',linewidth=2, label='int BLA')
plt.plot(mean_int_hpc_s.as_units('s'), color='b',linewidth=2, label='int HPC')
plt.legend()

In [102]:
plt.figure(figsize=(12,6))

bk.plot.intervals(states['Rem'])
bk.plot.intervals(states['sws'], col='grey')
plt.plot(mean_sync_pyr_bla_s.as_units('s'), color='g',linewidth=2, label='synchrony BLA')
plt.plot(mean_sync_pyr_hpc_s.as_units('s'), color='r',linewidth=2, label='synchrony Hpc')
plt.legend()
plt.savefig('C:/Users/maell/Documents/ENS/Cours/M2/S1/TINS/Figure/synchrony_across_time.svg')

# Tests 

## To compute figure of transitions from rem to sws or sws to rem 

In [5]:
def transitions_rem_sws(path):
    '''
    MC 10/01/22
    Inputs:
    
    Outputs:
    '''
    
    mean = []
    nb_sessions_considered = 0
    nb_transitions = 0 
    fig, ax = plt.subplots()
    means =[]

    nb_sessions_considered += 1
    mean_sync_pyr_bla = mean_synchrony(neurons_pyr_bla, window=1, smallbins=0.1, shift=1)
    mean_sync_pyr_bla_s = bk.compute.nts_smooth(mean_sync_pyr_bla, 100,50)

    transitions = bk.compute.transitions_times(states)
    transitions_rem_sws = transitions[1][('Rem', 'sws')].index.values

    nb_transitions += len(transitions_rem_sws)
    for i  in transitions_rem_sws:
        interval = nts.IntervalSet(i-20_000_000, i+ 40_000_000, time_units='us')
        m = mean_sync_pyr_bla.restrict(interval).values
        means.append(m)
        mean.append(m)
    #plt.plot(m, color = 'grey', alpha = 0.3)
    m = (np.nanmean(means, axis=0))
    plt.plot(m, color = 'grey')
    a = mpatches.Rectangle((0,0), 20, 0.5, facecolor = 'orange', alpha = 0.5)
    ax.add_patch(a)
    overall_mean = (np.nanmean(mean, axis=0))
    plt.plot(overall_mean, color = 'red', linewidth = 3)
    plt.title(f'transitions sws to rem, rat 08, #Transitions = {nb_transitions}, #Sessions = {nb_sessions_considered}')


In [88]:
rats = ['Rat08', 'Rat09','Rat10', 'Rat11']
to_ignore = ['Rat08-20130720', 
             'Rat08-20130722', 
             'Rat08-20130718',
             "Rat09-20140408",
             "Rat09-20140409",
             "Rat10-20140619",
             "Rat10-20140620",
             "Rat10-20140622",
             "Rat10-20140708",
             "Rat11-20150309",
             "Rat11-20150318",
             "Rat11-20150319", 
             "Rat11-20150324", 
             "Rat11-20150401"]
mean = []
nb_sessions_considered = 0
nb_transitions = 0 
fig, ax = plt.subplots()

for rat in rats:
    os.chdir(f'E:/DATA/GG-Dataset/{rat}/')
    means =[]
    
    for i in os.listdir(f'E:/DATA/GG-Dataset/{rat}/'):
        if i.endswith('.mat') or i in to_ignore:
            continue

        local_path= f'{rat}/{i}'
        print(local_path)
        bk.load.current_session_linux(base_folder= 'E:/DATA/GG-Dataset/',local_path= local_path)
        states = bk.load.states() 
        if states['Rem'].tot_length()/1_000_000 < 100 or states['sws'].tot_length()/1_000_000 < 100: 
            continue
        neurons,metadata = bk.load.spikes() 

        neurons_pyr_bla = neurons[(metadata['Type'] == 'Pyr') & (metadata['Region'] == 'BLA')]
        if len(neurons_pyr_bla) < 20: 
            continue
        nb_sessions_considered += 1
        mean_sync_pyr_bla = mean_synchrony(neurons_pyr_bla, window=1, smallbins=0.1, shift=1)
        mean_sync_pyr_bla_s = bk.compute.nts_smooth(mean_sync_pyr_bla, 100,50)

        transitions = bk.compute.transitions_times(states)
        transitions_rem_sws = transitions[1][('Rem', 'sws')].index.values
        
        nb_transitions += len(transitions_rem_sws)
        for i  in transitions_rem_sws:
            interval = nts.IntervalSet(i-20_000_000, i+ 40_000_000, time_units='us')
            m = mean_sync_pyr_bla.restrict(interval).values
            means.append(m)
            mean.append(m)
            #plt.plot(m, color = 'grey', alpha = 0.3)
    m = (np.nanmean(means, axis=0))
    plt.plot(m, color = 'grey')
a = mpatches.Rectangle((0,0), 20, 0.5, facecolor = 'orange', alpha = 0.5)
ax.add_patch(a)
overall_mean = (np.nanmean(mean, axis=0))
plt.plot(overall_mean, color = 'red', linewidth = 3)
plt.title(f'transitions sws to rem, rat 08, #Transitions = {nb_transitions}, #Sessions = {nb_sessions_considered}')


Rat08/Rat08-20130713
Rat : 8 on day : 6
Working with session Rat08-20130713 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130713
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy


  0%|                                                                                        | 0/22679 [00:00<?, ?it/s]C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
100%|███████████████████████████████████████████████████████████████████████████| 22679/22679 [00:24<00:00, 918.77it/s]


Rat08/Rat08-20130708
Rat : 8 on day : 1
Working with session Rat08-20130708 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130708
Data already saved in Numpy format, loading them from here:
Rat08-20130708-neurons.npy
Rat08-20130708-metadata.npy
Rat08/Rat08-20130709
Rat : 8 on day : 2
Working with session Rat08-20130709 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130709
Data already saved in Numpy format, loading them from here:
Rat08-20130709-neurons.npy
Rat08-20130709-metadata.npy


 21%|███████████████▋                                                           | 5326/25510 [00:04<00:17, 1176.75it/s]C:\Users\maell\AppData\Local\Temp/ipykernel_23996/1913985095.py:31: RuntimeWarning: Mean of empty slice
  mean_sync[i] = np.nanmean(corrcoef) #compute the mean of corrcoef for each window, disregarding the nan
100%|██████████████████████████████████████████████████████████████████████████| 25510/25510 [00:22<00:00, 1119.75it/s]


Rat08/Rat08-20130710
Rat : 8 on day : 3
Working with session Rat08-20130710 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130710
Data already saved in Numpy format, loading them from here:
Rat08-20130710-neurons.npy
Rat08-20130710-metadata.npy


...

 24%|██████████████████▏                                                        | 5431/22385 [00:04<00:15, 1094.12it/s]C:\Users\maell\AppData\Local\Temp/ipykernel_23996/1913985095.py:31: RuntimeWarning: Mean of empty slice
  mean_sync[i] = np.nanmean(corrcoef) #compute the mean of corrcoef for each window, disregarding the nan
100%|██████████████████████████████████████████████████████████████████████████| 22385/22385 [00:18<00:00, 1218.03it/s]


Rat11/Rat11-20150331
Rat : 11 on day : 19
Working with session Rat11-20150331 @ E:/DATA/GG-Dataset/Rat11/Rat11-20150331
Data already saved in Numpy format, loading them from here:
Rat11-20150331-neurons.npy
Rat11-20150331-metadata.npy


100%|██████████████████████████████████████████████████████████████████████████| 22776/22776 [00:13<00:00, 1627.67it/s]


Rat11/Rat11-20150402
Rat : 11 on day : 21
Working with session Rat11-20150402 @ E:/DATA/GG-Dataset/Rat11/Rat11-20150402
Data already saved in Numpy format, loading them from here:
Rat11-20150402-neurons.npy
Rat11-20150402-metadata.npy
Rat11/Rat11-20150403
Rat : 11 on day : 22
Working with session Rat11-20150403 @ E:/DATA/GG-Dataset/Rat11/Rat11-20150403
Data already saved in Numpy format, loading them from here:
Rat11-20150403-neurons.npy
Rat11-20150403-metadata.npy


Text(0.5, 1.0, 'transitions sws to rem, rat 08, #Transitions = 135, #Sessions = 28')

In [100]:
means =[]
nb_sessions_considered = 0
fig, ax = plt.subplots()
for i in os.listdir('E:/DATA/GG-Dataset/Rat08/'):
    if i.endswith('.mat') or i == 'Rat08-20130720' or  i == 'Rat08-20130722' or  i == 'Rat08-20130718':
        continue
        
    local_path= f'Rat08/{i}'
    bk.load.current_session_linux(base_folder= 'E:/DATA/GG-Dataset/',local_path= local_path)
    states = bk.load.states() 
    neurons,metadata = bk.load.spikes() 
    
    neurons_pyr_bla = neurons[(metadata['Type'] == 'Pyr') & (metadata['Region'] == 'BLA')]
    if len(neurons_pyr_bla) < 30: 
        continue
    nb_sessions_considered += 1
    mean_sync_pyr_bla = mean_synchrony(neurons_pyr_bla, window=1, smallbins=0.1, shift=1)
    mean_sync_pyr_bla_s = bk.compute.nts_smooth(mean_sync_pyr_bla, 100,50)
    
    transitions = bk.compute.transitions_times(states)
    transitions_rem_sws = transitions[1][('Rem', 'sws')].index.values
    transitions_sws_rem = transitions[1][('sws', 'Rem')].index.values

    for i  in transitions_rem_sws:
        interval = nts.IntervalSet(i-20_000_000, i+ 40_000_000, time_units='us')
        m = mean_sync_pyr_bla.restrict(interval).values
        means.append(m)
        plt.plot(m, color = 'grey', alpha = 0.3)
a = mpatches.Rectangle((0,0), 20, 1, facecolor = 'orange', alpha = 0.5)
ax.add_patch(a)
mean = np.nanmean(means, axis=0)
plt.plot(mean, color = 'r', linewidth=4)
plt.title(f'transitions rem to sws, rat 08, #Transitions = {len(means)}, #Sessions = {nb_sessions_considered}')

Rat : 8 on day : 6
Working with session Rat08-20130713 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130713
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy


  0%|                                                                                        | 0/22679 [00:00<?, ?it/s]C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
100%|██████████████████████████████████████████████████████████████████████████| 22679/22679 [00:17<00:00, 1275.00it/s]


Rat : 8 on day : 1
Working with session Rat08-20130708 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130708
Data already saved in Numpy format, loading them from here:
Rat08-20130708-neurons.npy
Rat08-20130708-metadata.npy
Rat : 8 on day : 2
Working with session Rat08-20130709 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130709
Data already saved in Numpy format, loading them from here:
Rat08-20130709-neurons.npy
Rat08-20130709-metadata.npy


  0%|                                                                                        | 0/25510 [00:00<?, ?it/s]C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
 21%|███████████████▌                                                           | 5299/25510 [00:04<00:14, 1435.76it/s]C:\Users\maell\AppData\Local\Temp/ipykernel_25084/453570434.py:31: RuntimeWarning: Mean of empty slice
  mean_sync[i] = np.nanmean(corrcoef) #compute the mean of corrcoef for each window, disregarding the nan
100%|██████████████████████████████████████████████████████████████████████████| 25510/25510 [00:18<00:00, 1364.02it/s]


Rat : 8 on day : 3
Working with session Rat08-20130710 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130710
Data already saved in Numpy format, loading them from here:
Rat08-20130710-neurons.npy
Rat08-20130710-metadata.npy


...

  0%|                                                                                        | 0/25112 [00:00<?, ?it/s]C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
 15%|███████████                                                                | 3689/25112 [00:01<00:09, 2300.06it/s]C:\Users\maell\AppData\Local\Temp/ipykernel_25084/453570434.py:31: RuntimeWarning: Mean of empty slice
  mean_sync[i] = np.nanmean(corrcoef) #compute the mean of corrcoef for each window, disregarding the nan
100%|██████████████████████████████████████████████████████████████████████████| 25112/25112 [00:12<00:00, 2039.16it/s]


Rat : 8 on day : 9
Working with session Rat08-20130717 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130717
Data already saved in Numpy format, loading them from here:
Rat08-20130717-neurons.npy
Rat08-20130717-metadata.npy


  0%|                                                                                        | 0/23243 [00:00<?, ?it/s]C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
100%|██████████████████████████████████████████████████████████████████████████| 23243/23243 [00:12<00:00, 1918.04it/s]


Rat : 8 on day : 11
Working with session Rat08-20130719 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130719
Data already saved in Numpy format, loading them from here:
Rat08-20130719-neurons.npy
Rat08-20130719-metadata.npy


Text(0.5, 1.0, 'transitions rem to sws, rat 08, #Transitions = 48, #Sessions = 8')

## boxplot of the synchrony around transitions 

In [13]:
rats = ['Rat08', 'Rat09','Rat10', 'Rat11']
to_ignore = ['Rat08-20130720', 
             'Rat08-20130722', 
             'Rat08-20130718',
             "Rat09-20140408",
             "Rat09-20140409",
             "Rat10-20140619",
             "Rat10-20140620",
             "Rat10-20140622",
             "Rat10-20140708",
             "Rat11-20150309",
             "Rat11-20150318",
             "Rat11-20150318",
             "Rat11-20150319", 
             "Rat11-20150324", 
             "Rat11-20150401"]

trans = {'Rem before transition': [], 
        'Rem after transition': [],
        'sws before transition' : [], 
        'sws after transition': []}

for rat in rats:
    print(rat)
    os.chdir(f'E:/DATA/GG-Dataset/{rat}/')
    means =[]
    
    for i in os.listdir(f'E:/DATA/GG-Dataset/{rat}/'):
        if i.endswith('.mat') or i in to_ignore:
            continue
        
        local_path= f'{rat}/{i}'
        bk.load.current_session_linux(base_folder= 'E:/DATA/GG-Dataset/',local_path= local_path)
        states = bk.load.states() 
        if states['Rem'].tot_length()/1_000_000 < 100 or states['sws'].tot_length()/1_000_000 < 100: 
            continue
        neurons,metadata = bk.load.spikes() 

        neurons_pyr_bla = neurons[(metadata['Type'] == 'Pyr') & (metadata['Region'] == 'BLA')]
        if len(neurons_pyr_bla) < 20: 
            continue
            
        mean_sync = mean_synchrony(neurons_pyr_bla)
        mean_sync = bk.compute.nts_smooth(mean_sync, 100,50)
        
        transitions = bk.compute.transitions_times(states)
        transitions_rem_sws = transitions[1][('Rem', 'sws')].index.values
        transitions_sws_rem = transitions[1][('sws', 'Rem')].index.values

        for i  in transitions_rem_sws:
            interval_rem = nts.IntervalSet(i-10_000_000, i, time_units='us')
            trans['Rem before transition'].append(np.mean(mean_sync.restrict(interval_rem).values))
            interval_sws = nts.IntervalSet(i, i+10_000_000, time_units='us')
            trans['sws after transition'].append(np.mean(mean_sync.restrict(interval_sws).values))

        for i  in transitions_sws_rem:
            interval_sws = nts.IntervalSet(i-10_000_000, i, time_units='us')
            trans['sws before transition'].append(np.mean(mean_sync.restrict(interval_sws).values))
            interval_rem = nts.IntervalSet(i, i+10_000_000, time_units='us')
            trans['Rem after transition'].append(np.mean(mean_sync.restrict(interval_rem).values))

trans = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in trans.items() ]))

Rat08
Rat : 8 on day : 6
Working with session Rat08-20130713 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130713
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy


  0%|                                                                                        | 0/22679 [00:00<?, ?it/s]C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
100%|██████████████████████████████████████████████████████████████████████████| 22679/22679 [00:13<00:00, 1703.56it/s]


Rat : 8 on day : 1
Working with session Rat08-20130708 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130708
Data already saved in Numpy format, loading them from here:
Rat08-20130708-neurons.npy
Rat08-20130708-metadata.npy
Rat : 8 on day : 2
Working with session Rat08-20130709 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130709
Data already saved in Numpy format, loading them from here:
Rat08-20130709-neurons.npy
Rat08-20130709-metadata.npy


 21%|███████████████▊                                                           | 5360/25510 [00:03<00:13, 1524.39it/s]C:\Users\maell\AppData\Local\Temp/ipykernel_24892/1913985095.py:31: RuntimeWarning: Mean of empty slice
  mean_sync[i] = np.nanmean(corrcoef) #compute the mean of corrcoef for each window, disregarding the nan
100%|██████████████████████████████████████████████████████████████████████████| 25510/25510 [00:15<00:00, 1602.63it/s]


Rat : 8 on day : 3
Working with session Rat08-20130710 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130710
Data already saved in Numpy format, loading them from here:
Rat08-20130710-neurons.npy
Rat08-20130710-metadata.npy


...

Rat : 11 on day : 18
Working with session Rat11-20150330 @ E:/DATA/GG-Dataset/Rat11/Rat11-20150330
Data already saved in Numpy format, loading them from here:
Rat11-20150330-neurons.npy
Rat11-20150330-metadata.npy


100%|██████████████████████████████████████████████████████████████████████████| 22385/22385 [00:09<00:00, 2434.10it/s]


Rat : 11 on day : 19
Working with session Rat11-20150331 @ E:/DATA/GG-Dataset/Rat11/Rat11-20150331
Data already saved in Numpy format, loading them from here:
Rat11-20150331-neurons.npy
Rat11-20150331-metadata.npy


100%|██████████████████████████████████████████████████████████████████████████| 22776/22776 [00:10<00:00, 2167.59it/s]


Rat : 11 on day : 21
Working with session Rat11-20150402 @ E:/DATA/GG-Dataset/Rat11/Rat11-20150402
Data already saved in Numpy format, loading them from here:
Rat11-20150402-neurons.npy
Rat11-20150402-metadata.npy
Rat : 11 on day : 22
Working with session Rat11-20150403 @ E:/DATA/GG-Dataset/Rat11/Rat11-20150403
Data already saved in Numpy format, loading them from here:
Rat11-20150403-neurons.npy
Rat11-20150403-metadata.npy


In [15]:
sns.violinplot(data = trans)

<AxesSubplot:>

In [13]:
rats = ['Rat08', 'Rat09','Rat10', 'Rat11']
to_ignore = ['Rat08-20130720', 
             'Rat08-20130722', 
             'Rat08-20130718',
             "Rat09-20140408",
             "Rat09-20140409",
             "Rat10-20140619",
             "Rat10-20140620",
             "Rat10-20140622",
             "Rat10-20140708",
             "Rat11-20150309",
             "Rat11-20150318",
             "Rat11-20150318",
             "Rat11-20150319", 
             "Rat11-20150324", 
             "Rat11-20150401"]


transitions = {'Rem': [], 'sws': []}

for rat in rats:
    print(rat)
    os.chdir(f'E:/DATA/GG-Dataset/{rat}/')
    means =[]
    
    for i in os.listdir(f'E:/DATA/GG-Dataset/{rat}/'):
        if i.endswith('.mat') or i in to_ignore:
            continue
        
        local_path= f'{rat}/{i}'
        bk.load.current_session_linux(base_folder= 'E:/DATA/GG-Dataset/',local_path= local_path)
        states = bk.load.states() 
        if states['Rem'].tot_length()/1_000_000 < 100 or states['sws'].tot_length()/1_000_000 < 100: 
            continue
            
        trans = {'Rem': [], 'sws': []}
        
        neurons,metadata = bk.load.spikes() 

        neurons_pyr_bla = neurons[(metadata['Type'] == 'Pyr') & (metadata['Region'] == 'BLA')]
        if len(neurons_pyr_bla) < 20: 
            continue
            
        mean_sync = mean_synchrony(neurons_pyr_bla)
        mean_sync = bk.compute.nts_smooth(mean_sync, 100,50)

        for i, j  in zip(states['Rem'].as_units('s').start, states['Rem'].as_units('s').end):
            interval = nts.IntervalSet(i,j, time_units='s')
            trans['Rem'].append(np.mean(mean_sync.restrict(interval).values))

        for i, j  in zip(states['sws'].as_units('s').start, states['sws'].as_units('s').end):
            interval = nts.IntervalSet(i,j, time_units='s')
            trans['sws'].append(np.mean(mean_sync.restrict(interval).values))
        
        transitions['Rem'].append(np.nanmean(trans['Rem']))
        transitions['sws'].append(np.nanmean(trans['sws']))


transitions = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in transitions.items() ]))

Rat08
Rat : 8 on day : 6
Working with session Rat08-20130713 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130713
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy


  0%|                                                                                        | 0/22679 [00:00<?, ?it/s]C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
100%|██████████████████████████████████████████████████████████████████████████| 22679/22679 [00:12<00:00, 1807.23it/s]


Rat : 8 on day : 1
Working with session Rat08-20130708 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130708
Data already saved in Numpy format, loading them from here:
Rat08-20130708-neurons.npy
Rat08-20130708-metadata.npy
Rat : 8 on day : 2
Working with session Rat08-20130709 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130709
Data already saved in Numpy format, loading them from here:
Rat08-20130709-neurons.npy
Rat08-20130709-metadata.npy


 21%|███████████████▉                                                           | 5423/25510 [00:04<00:14, 1392.89it/s]C:\Users\maell\AppData\Local\Temp/ipykernel_42528/1913985095.py:31: RuntimeWarning: Mean of empty slice
  mean_sync[i] = np.nanmean(corrcoef) #compute the mean of corrcoef for each window, disregarding the nan
100%|██████████████████████████████████████████████████████████████████████████| 25510/25510 [00:18<00:00, 1392.42it/s]


Rat : 8 on day : 3
Working with session Rat08-20130710 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130710
Data already saved in Numpy format, loading them from here:
Rat08-20130710-neurons.npy
Rat08-20130710-metadata.npy


...

Rat : 11 on day : 18
Working with session Rat11-20150330 @ E:/DATA/GG-Dataset/Rat11/Rat11-20150330
Data already saved in Numpy format, loading them from here:
Rat11-20150330-neurons.npy
Rat11-20150330-metadata.npy


100%|██████████████████████████████████████████████████████████████████████████| 22385/22385 [00:10<00:00, 2135.12it/s]


Rat : 11 on day : 19
Working with session Rat11-20150331 @ E:/DATA/GG-Dataset/Rat11/Rat11-20150331
Data already saved in Numpy format, loading them from here:
Rat11-20150331-neurons.npy
Rat11-20150331-metadata.npy


100%|██████████████████████████████████████████████████████████████████████████| 22776/22776 [00:12<00:00, 1880.72it/s]


Rat : 11 on day : 21
Working with session Rat11-20150402 @ E:/DATA/GG-Dataset/Rat11/Rat11-20150402
Data already saved in Numpy format, loading them from here:
Rat11-20150402-neurons.npy
Rat11-20150402-metadata.npy
Rat : 11 on day : 22
Working with session Rat11-20150403 @ E:/DATA/GG-Dataset/Rat11/Rat11-20150403
Data already saved in Numpy format, loading them from here:
Rat11-20150403-neurons.npy
Rat11-20150403-metadata.npy


In [14]:
1 +1

2

In [23]:
m_rem = np.nanmean(transitions['Rem'])
m_sws = np.nanmean(transitions['sws'])
std_rem = np.std(transitions['Rem'])
std_sws = np.std(transitions['sws'])

for i,j in zip(transitions['Rem'], transitions['sws']):
    plt.plot([0,1], [i,j], '.-', color = 'grey', alpha = 0.3)
    
plt.errorbar([0,1], [m_rem,m_sws], yerr= [std_rem, std_sws],  color = 'black', linewidth= 5)

<ErrorbarContainer object of 3 artists>

In [19]:
std_rem

0.0393530078370237

In [36]:
rats = ['Rat08', 'Rat09','Rat10', 'Rat11']
to_ignore = ['Rat08-20130720', 
             'Rat08-20130722', 
             'Rat08-20130718',
             "Rat09-20140408",
             "Rat09-20140409",
             "Rat10-20140619",
             "Rat10-20140620",
             "Rat10-20140622",
             "Rat10-20140708",
             "Rat11-20150309",
             "Rat11-20150318",
             "Rat11-20150318",
             "Rat11-20150319", 
             "Rat11-20150324", 
             "Rat11-20150401"]

trans = {'Rem before transition': [], 
        'Rem after transition': [],
         'Rem': [],
        'sws before transition' : [], 
        'sws after transition': [],
        'sws': []}

for rat in rats:
    print(rat)
    os.chdir(f'E:/DATA/GG-Dataset/{rat}/')
    means =[]
    
    for i in os.listdir(f'E:/DATA/GG-Dataset/{rat}/'):
        if i.endswith('.mat') or i in to_ignore:
            continue
        
        local_path= f'{rat}/{i}'
        bk.load.current_session_linux(base_folder= 'E:/DATA/GG-Dataset/',local_path= local_path)
        states = bk.load.states() 
        if states['Rem'].tot_length()/1_000_000 < 100 or states['sws'].tot_length()/1_000_000 < 100: 
            continue
        neurons,metadata = bk.load.spikes() 

        neurons_pyr_bla = neurons[(metadata['Type'] == 'Pyr') & (metadata['Region'] == 'BLA')]
        if len(neurons_pyr_bla) < 20: 
            continue
            
        mean_sync = mean_synchrony(neurons_pyr_bla)
        mean_sync = bk.compute.nts_smooth(mean_sync, 100,50)
        
        transitions = bk.compute.transitions_times(states)
        transitions_rem_sws = transitions[1][('Rem', 'sws')].index.values
        transitions_sws_rem = transitions[1][('sws', 'Rem')].index.values

        for i  in transitions_rem_sws:
            interval_rem = nts.IntervalSet(i-10_000_000, i, time_units='us')
            trans['Rem before transition'].append(np.mean(mean_sync.restrict(interval_rem).values))
            interval_sws = nts.IntervalSet(i, i+10_000_000, time_units='us')
            trans['sws after transition'].append(np.mean(mean_sync.restrict(interval_sws).values))

        for i  in transitions_sws_rem:
            interval_sws = nts.IntervalSet(i-10_000_000, i, time_units='us')
            trans['sws before transition'].append(np.mean(mean_sync.restrict(interval_sws).values))
            interval_rem = nts.IntervalSet(i, i+10_000_000, time_units='us')
            trans['Rem after transition'].append(np.mean(mean_sync.restrict(interval_rem).values))

        for i, j  in zip(states['Rem'].as_units('s').start, states['Rem'].as_units('s').end):
            interval = nts.IntervalSet(i,j, time_units='s')
            trans['Rem'].append(np.mean(mean_sync.restrict(interval).values))

        for i, j  in zip(states['sws'].as_units('s').start, states['sws'].as_units('s').end):
            interval = nts.IntervalSet(i,j, time_units='s')
            trans['sws'].append(np.mean(mean_sync.restrict(interval).values))

trans = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in trans.items() ]))

Rat08
Rat : 8 on day : 6
Working with session Rat08-20130713 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130713
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy


  0%|                                                                                        | 0/22679 [00:00<?, ?it/s]C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
100%|██████████████████████████████████████████████████████████████████████████| 22679/22679 [00:12<00:00, 1878.88it/s]


Rat : 8 on day : 1
Working with session Rat08-20130708 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130708
Data already saved in Numpy format, loading them from here:
Rat08-20130708-neurons.npy
Rat08-20130708-metadata.npy
Rat : 8 on day : 2
Working with session Rat08-20130709 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130709
Data already saved in Numpy format, loading them from here:
Rat08-20130709-neurons.npy
Rat08-20130709-metadata.npy


 21%|███████████████▉                                                           | 5411/25510 [00:02<00:09, 2172.15it/s]C:\Users\maell\AppData\Local\Temp/ipykernel_20972/1913985095.py:31: RuntimeWarning: Mean of empty slice
  mean_sync[i] = np.nanmean(corrcoef) #compute the mean of corrcoef for each window, disregarding the nan
100%|██████████████████████████████████████████████████████████████████████████| 25510/25510 [00:15<00:00, 1675.45it/s]


Rat : 8 on day : 3
Working with session Rat08-20130710 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130710
Data already saved in Numpy format, loading them from here:
Rat08-20130710-neurons.npy
Rat08-20130710-metadata.npy


...

Rat : 11 on day : 18
Working with session Rat11-20150330 @ E:/DATA/GG-Dataset/Rat11/Rat11-20150330
Data already saved in Numpy format, loading them from here:
Rat11-20150330-neurons.npy
Rat11-20150330-metadata.npy


100%|██████████████████████████████████████████████████████████████████████████| 22385/22385 [00:06<00:00, 3326.42it/s]


Rat : 11 on day : 19
Working with session Rat11-20150331 @ E:/DATA/GG-Dataset/Rat11/Rat11-20150331
Data already saved in Numpy format, loading them from here:
Rat11-20150331-neurons.npy
Rat11-20150331-metadata.npy


100%|██████████████████████████████████████████████████████████████████████████| 22776/22776 [00:06<00:00, 3574.29it/s]


Rat : 11 on day : 21
Working with session Rat11-20150402 @ E:/DATA/GG-Dataset/Rat11/Rat11-20150402
Data already saved in Numpy format, loading them from here:
Rat11-20150402-neurons.npy
Rat11-20150402-metadata.npy
Rat : 11 on day : 22
Working with session Rat11-20150403 @ E:/DATA/GG-Dataset/Rat11/Rat11-20150403
Data already saved in Numpy format, loading them from here:
Rat11-20150403-neurons.npy
Rat11-20150403-metadata.npy


In [38]:
trans

,Rem before transition,Rem after transition,Rem,sws before transition,sws after transition,sws
0,0.051081,0.050926,0.051378,0.051466,0.050442,0.055960
1,0.051700,0.050512,0.050401,0.051024,0.050501,0.054073
2,0.053376,0.056376,0.051836,0.057697,0.054289,0.057971
3,0.063059,0.053338,0.051481,0.054054,0.060635,0.061147
4,0.052149,0.050869,0.051300,0.050258,0.054085,0.058583
...,...,...,...,...,...,...
616,NaN,NaN,NaN,NaN,NaN,NaN
617,NaN,NaN,NaN,NaN,NaN,0.180600
618,NaN,NaN,NaN,NaN,NaN,NaN
619,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
sp.stats.kruskal(trans['sws'], trans['Rem'], nan_policy='omit')

KruskalResult(statistic=2.6222440264796205, pvalue=0.1053753245150037)

In [23]:
t = synchrony_around_transitions(transitions, mean_sync_pyr_bla_s)

In [13]:
sns.boxplot(data = trans, whis=np.inf)
sns.swarmplot(data = trans, color=".2")
#plt.savefig('C:/Users/maell/Documents/ENS/Cours/M2/S1/TINS/Figure/means_synchrony_rem_sws.svg')

C:\Users\maell\anaconda3\lib\site-packages\seaborn\categorical.py:1296: UserWarning: 14.8% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
  warnings.warn(msg, UserWarning)
C:\Users\maell\anaconda3\lib\site-packages\seaborn\categorical.py:1296: UserWarning: 17.1% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
  warnings.warn(msg, UserWarning)
C:\Users\maell\anaconda3\lib\site-packages\seaborn\categorical.py:1296: UserWarning: 15.1% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
  warnings.warn(msg, UserWarning)
C:\Users\maell\anaconda3\lib\site-packages\seaborn\categorical.py:1296: UserWarning: 46.2% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
  warnings.warn(msg, UserWarning)


<AxesSubplot:>

In [51]:
means =[]
fig, ax = plt.subplots()
for i  in transitions_sws_rem:
    interval = nts.IntervalSet(i-150_000_000, i+ 50_000_000, time_units='us')
    m = mean_sync_pyr_bla.restrict(interval).values
    means.append(m)
    plt.plot(m, color = 'grey', alpha = 0.3)
a = mpatches.Rectangle((150,0), 50, 0.3, facecolor = 'orange', alpha = 0.5)
ax.add_patch(a)
mean = np.mean(means, axis=0)
plt.plot(mean, color = 'r', linewidth=4)

In [156]:
trans = {'Rem before transition': [], 
        'Rem after transition': [],
         'Rem': [],
        'sws before transition' : [], 
        'sws after transition': [],
        'sws': []}
for i  in transitions_rem_sws:
    interval_rem = nts.IntervalSet(i-10_000_000, i, time_units='s')
    trans['Rem before transition'].append(np.mean(mean_sync_pyr_bla.restrict(interval_rem).values))
    interval_sws = nts.IntervalSet(i, i+10_000_000, time_units='s')
    trans['sws after transition'].append(np.mean(mean_sync_pyr_hpc.restrict(interval_sws).values))
    
for i  in transitions_sws_rem:
    interval_sws = nts.IntervalSet(i-10_000_000, i, time_units='s')
    trans['sws before transition'].append(np.mean(mean_sync_pyr_bla.restrict(interval_sws).values))
    interval_rem = nts.IntervalSet(i, i+10_000_000, time_units='s')
    trans['Rem after transition'].append(np.mean(mean_sync_pyr_hpc.restrict(interval_rem).values))
    
for i, j  in zip(states['Rem'].as_units('s').start, states['Rem'].as_units('s').end):
    interval = nts.IntervalSet(i,j, time_units='s')
    trans['Rem'].append(np.mean(mean_sync_pyr_bla.restrict(interval).values))
    
for i, j  in zip(states['sws'].as_units('s').start, states['sws'].as_units('s').end):
    interval = nts.IntervalSet(i,j, time_units='s')
    trans['sws'].append(np.mean(mean_sync_pyr_bla.restrict(interval).values))

C:\Users\maell\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\maell\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## Get the distribution of the Pearson's correlation coefficients 

In [27]:
rats = ['Rat08', 'Rat09', 'Rat10', 'Rat11']
to_ignore = ['Rat08-20130720', 
             'Rat08-20130722', 
             'Rat08-20130718',
             "Rat09-20140408",
             "Rat09-20140409",
             "Rat10-20140619",
             "Rat10-20140620",
             "Rat10-20140622",
             "Rat10-20140708",
             "Rat11-20150309",
             "Rat11-20150318",
             "Rat11-20150319", 
             "Rat11-20150324",
             "Rat11-20150327",
             "Rat11-20150401"]

corr_coeffs = {'Corr_rem': [], 'Corr_sws': []}

for rat in rats:
    os.chdir(f'E:/DATA/GG-Dataset/{rat}/')
    
    for i in os.listdir(f'E:/DATA/GG-Dataset/{rat}/'):
        if i.endswith('.mat') or i in to_ignore:
            continue

        local_path= f'{rat}/{i}'
        print(local_path)
        bk.load.current_session_linux(base_folder= 'E:/DATA/GG-Dataset/',local_path= local_path)
        states = bk.load.states()
        neurons,metadata = bk.load.spikes() 
        
        print(f"total duration Rem {states['Rem'].tot_length()/1_000_000}")
        
        if states['Rem'].tot_length()/1_000_000 < 100:
            print(f'{local_path} NOT ENOUGH REM')
            continue
            
        

        neurons_pyr_bla = neurons[(metadata['Type'] == 'Pyr') & (metadata['Region'] == 'BLA')]
        if len(neurons_pyr_bla) < 20:
            print('NOT ENOUGH NEURONS')
            continue
        
        mean_sync_pyr_bla = mean_synchrony(neurons_pyr_bla, window=1, smallbins=0.1, shift=1)
        mean_sync_pyr_bla_s = bk.compute.nts_smooth(mean_sync_pyr_bla, 100,50)
        
        mean_fr_pyr_bla = mean_firing_rate(neurons_pyr_bla, window=1, smallbins=0.1, shift=1)
        mean_fr_pyr_bla_s = bk.compute.nts_smooth(mean_fr_pyr_bla, 100,50)
        
        fr_rem = mean_fr_pyr_bla_s.restrict(states['Rem']).values
        sync_rem = mean_sync_pyr_bla_s.restrict(states['Rem']).values
        
        fr_sws = mean_fr_pyr_bla_s.restrict(states['sws']).values
        sync_sws = mean_sync_pyr_bla_s.restrict(states['sws']).values
        
        sync_rem, fr_rem,_ = corr_coeff_without_nan(sync_rem, fr_rem)
        sync_sws, fr_sws,_ = corr_coeff_without_nan(sync_sws, fr_sws)
        
        corr_rem = np.corrcoef(sync_rem, fr_rem)
        corr_sws = np.corrcoef(sync_sws, fr_sws)

        corr_coeffs['Corr_rem'].append(corr_rem[0,1])
        corr_coeffs['Corr_sws'].append(corr_sws[0,1])
        
        '''plt.figure()
        g = sns.regplot(mean_fr_pyr_bla_s.restrict(states['Rem']).values, 
                        mean_sync_pyr_bla_s.restrict(states['Rem']).values, 
                        scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, x_ci='sd', color='orange')
        sns.regplot(mean_fr_pyr_bla_s.restrict(states['sws']).values, 
                    mean_sync_pyr_bla_s.restrict(states['sws']).values, 
                    scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, x_ci='sd', color='grey')
        g.set(xlabel ='mean firing rate', ylabel = 'mean synchrony',
              title='Correlation between synchrony and firing rate in BLA interneurons')
        g.legend(labels=['Rem', 'sws'])
        
        plt.title(f'Session {local_path}, Pearson coeff REM = {round(corr_rem[0,1],2)}, Pearson coeff NREM = {round(corr_sws[0,1],2)}')'''

Rat08/Rat08-20130713
Rat : 8 on day : 6
Working with session Rat08-20130713 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130713
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy
total duration Rem 2371.0


  0%|                                                                                        | 0/22679 [00:00<?, ?it/s]C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
100%|█████████████████████████████████████████████████████████████████████████| 22679/22679 [00:00<00:00, 37741.72it/s]


Rat08/Rat08-20130708
Rat : 8 on day : 1
Working with session Rat08-20130708 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130708
Data already saved in Numpy format, loading them from here:
Rat08-20130708-neurons.npy
Rat08-20130708-metadata.npy
total duration Rem 1604.0
NOT ENOUGH NEURONS
Rat08/Rat08-20130709
Rat : 8 on day : 2
Working with session Rat08-20130709 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130709
Data already saved in Numpy format, loading them from here:
Rat08-20130709-neurons.npy
Rat08-20130709-metadata.npy
total duration Rem 1930.0


 21%|███████████████▉                                                           | 5405/25510 [00:03<00:11, 1824.09it/s]C:\Users\maell\AppData\Local\Temp/ipykernel_20972/1913985095.py:31: RuntimeWarning: Mean of empty slice
  mean_sync[i] = np.nanmean(corrcoef) #compute the mean of corrcoef for each window, disregarding the nan
100%|█████████████████████████████████████████████████████████████████████████| 25510/25510 [00:00<00:00, 33316.89it/s]


Rat08/Rat08-20130710
Rat : 8 on day : 3
Working with session Rat08-20130710 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130710
Data already saved in Numpy format, loading them from here:
Rat08-20130710-neurons.npy
Rat08-20130710-metadata.npy
total duration Rem 1480.0


...

Rat11/Rat11-20150330
Rat : 11 on day : 18
Working with session Rat11-20150330 @ E:/DATA/GG-Dataset/Rat11/Rat11-20150330
Data already saved in Numpy format, loading them from here:
Rat11-20150330-neurons.npy
Rat11-20150330-metadata.npy
total duration Rem 1395.0


100%|█████████████████████████████████████████████████████████████████████████| 22385/22385 [00:01<00:00, 17584.55it/s]


Rat11/Rat11-20150331
Rat : 11 on day : 19
Working with session Rat11-20150331 @ E:/DATA/GG-Dataset/Rat11/Rat11-20150331
Data already saved in Numpy format, loading them from here:
Rat11-20150331-neurons.npy
Rat11-20150331-metadata.npy
total duration Rem 1375.0


100%|█████████████████████████████████████████████████████████████████████████| 22776/22776 [00:00<00:00, 24617.83it/s]


Rat11/Rat11-20150402
Rat : 11 on day : 21
Working with session Rat11-20150402 @ E:/DATA/GG-Dataset/Rat11/Rat11-20150402
Data already saved in Numpy format, loading them from here:
Rat11-20150402-neurons.npy
Rat11-20150402-metadata.npy
total duration Rem 528.0
NOT ENOUGH NEURONS
Rat11/Rat11-20150403
Rat : 11 on day : 22
Working with session Rat11-20150403 @ E:/DATA/GG-Dataset/Rat11/Rat11-20150403
Data already saved in Numpy format, loading them from here:
Rat11-20150403-neurons.npy
Rat11-20150403-metadata.npy
total duration Rem 612.0
NOT ENOUGH NEURONS


In [28]:
corr_coeffs

{'Corr_rem': [-0.7264069123691904,
  -0.8435611379902256,
  -0.6780721132984141,
  -0.7758978750930776,
  -0.6462165218318869,
  -0.6536431809028816,
  -0.8745686310654911,
  -0.40601354406275575,
  -0.6214593871604931,
  -0.7148295058620436,
  -0.6698219801211841,
  0.0741557396848366,
  -0.7038039243009637,
  -0.9039885654296196,
  -0.4920947080326685,
  -0.7604310015811306,
  -0.7707119855579531,
  -0.8365981064056348,
  -0.5997648147064497,
  -0.6650797890455,
  -0.7207414066669935,
  -0.8426170887474401,
  -0.5255321298755488,
  -0.7896903551001633,
  -0.7120091803055135,
  -0.615665414908355,
  -0.6172611614004304],
 'Corr_sws': [-0.6411961904789658,
  -0.7306636543212914,
  -0.7218411518171706,
  -0.6930897748900755,
  -0.6273814997687603,
  -0.8827246429205031,
  -0.7820741279607375,
  -0.7696600500384255,
  -0.6633607747931046,
  -0.7949447881057493,
  -0.7529874176104558,
  -0.5826728599292252,
  -0.5874323572260504,
  -0.7679813836738495,
  -0.7262433545135184,
  -0.79483345

In [32]:
len(corr_coeffs['Corr_rem'])

27

In [34]:
sp.stats.kruskal(corr_coeffs['Corr_rem'], corr_coeffs['Corr_sws'], nan_policy='omit')

KruskalResult(statistic=2.8451178451178407, pvalue=0.09165187345070036)

In [33]:
corr_coeffs = pd.DataFrame(corr_coeffs)
sns.boxplot(data = corr_coeffs, whis=np.inf)
sns.swarmplot(data = corr_coeffs, color=".2")
plt.ylim(-1, 0.2)

(-1.0, 0.2)

### Correlation between firing rate and synchrony

In [135]:
np.shape(mean_sync_pyr_bla_s.restrict(states['Rem']).values)

(1345,)

In [66]:
plt.figure()
plt.subplot(1,2,1)
g = sns.regplot(mean_fr_int_bla_s.restrict(states['Rem']).values, mean_sync_int_bla_s.restrict(states['Rem']).values, scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, x_ci='sd', color='orange')
sns.regplot(mean_fr_int_bla_s.restrict(states['sws']).values, mean_sync_int_bla_s.restrict(states['sws']).values, scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, x_ci='sd', color='grey')
sns.regplot(mean_fr_int_bla_s[100:-100].restrict(states['wake']).values, mean_sync_int_bla_s[100:-100].restrict(states['wake']).values, scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, x_ci='sd')
g.set(xlabel ='mean firing rate', ylabel = 'mean synchrony',
      title='Correlation between synchrony and firing rate in BLA interneurons')
g.legend(labels=['Rem', 'sws', 'Wake'])
plt.subplot(1,2,2)
g = sns.regplot(mean_fr_int_bla_s.restrict(states['Rem']).values, mean_sync_nan_int_bla_s.restrict(states['Rem']).values, scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, robust=True, x_ci='sd', color='orange')
sns.regplot(mean_fr_int_bla_s.restrict(states['sws']).values, mean_sync_nan_int_bla_s.restrict(states['sws']).values, scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, robust=True, x_ci='sd', color='grey')
sns.regplot(mean_fr_int_bla_s[100:-100].restrict(states['wake']).values, mean_sync_nan_int_bla_s[100:-100].restrict(states['wake']).values, scatter_kws={'alpha':0.1, 's':5}, robust=True, line_kws={'lw':5}, x_ci='sd')
g.set(xlabel ='mean firing rate', ylabel = 'mean synchrony (with nan to 0)',
      title='Correlation between synchrony (with nan to 0) and firing rate in BLA interneurons')
g.legend(labels=['Rem', 'sws', 'Wake'])

C:\Users\maell\anaconda3\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(
C:\Users\maell\anaconda3\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(
C:\Users\maell\anaconda3\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(
C:\Users\maell\anacond

In [96]:
plt.subplot(2,2,1)
plt.plot(mean_fr_pyr_hpc_s[100:-100].as_units('s'), mean_sync_pyr_hpc_s[100:-100].as_units('s'), '.')
plt.title('pyr hpc, Pearson coeff = '+str(corr_pyr_hpc[1,0]))
plt.subplot(2,2,2)
plt.plot(mean_fr_pyr_bla_s[100:-100], mean_sync_pyr_bla_s[100:-100], '.')
plt.title('pyr bla, Pearson coeff = '+str(corr_pyr_bla[1,0]))
plt.subplot(2,2,3)
plt.plot(mean_fr_int_hpc_s[100:-100], mean_sync_int_hpc_s[100:-100], '.')
plt.title('int hpc, Pearson coeff = '+str(corr_int_hpc[1,0]))
plt.subplot(2,2,4)
plt.plot(mean_fr_int_bla_s[100:-100], mean_sync_int_bla_s[100:-100], '.')
plt.title('int bla, Pearson coeff = '+str(corr_int_bla[1,0]))

Text(0.5, 1.0, 'int bla, Pearson coeff = -0.7551644915307446')

# Correlation between pourcent active neurons and mean firing rate

In [18]:
fr_rem = mean_fr_pyr_bla_s.restrict(states['Rem']).values
sync_rem = mean_sync_pyr_bla_s.restrict(states['Rem']).values

fr_sws = mean_fr_pyr_bla_s.restrict(states['sws']).values
sync_sws = mean_sync_pyr_bla_s.restrict(states['sws']).values

sync_rem, fr_rem,_ = corr_coeff_without_nan(sync_rem, fr_rem)
sync_sws, fr_sws,_ = corr_coeff_without_nan(sync_sws, fr_sws)

corr_rem = np.corrcoef(sync_rem, fr_rem)
corr_sws = np.corrcoef(sync_sws, fr_sws)

In [21]:
plt.figure()
g = sns.regplot(mean_fr_pyr_bla_s.restrict(states['Rem']).values, 
                mean_sync_pyr_bla_s.restrict(states['Rem']).values, 
                scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, x_ci='sd', color='orange')
sns.regplot(mean_fr_pyr_bla_s.restrict(states['sws']).values, 
            mean_sync_pyr_bla_s.restrict(states['sws']).values, 
            scatter_kws={'alpha':0.1, 's':5}, line_kws={'lw':5}, x_ci='sd', color='grey')
g.set(xlabel ='mean firing rate', ylabel = 'Mean synchrony (%)',
      title= f'REM Correlation {corr_rem[0,1]}, NREM Correlation {corr_sws[0,1]}')
g.legend(labels=['Rem', 'sws'])

C:\Users\maell\anaconda3\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(
C:\Users\maell\anaconda3\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(


In [258]:
plt.figure()

bk.plot.intervals(states['Rem'])
bk.plot.intervals(states['sws'], col='grey')
plt.plot(mean_fr_pyr_bla_s.as_units('s'), color='g', label = 'pyr bla')
plt.plot(mean_fr_int_bla_s.as_units('s'), color='k', label='int bla')
plt.title('Mean firing rates. \n Session: Rat08-20130713')
plt.xlabel('Time (s)')
plt.legend()
plt.text(-1000,0.42, 'Time bins: 1s \nSmoothing parameters: \n    Number of points in window: 100 \n    Standard deviation: 50')
plt.figtext(0.12, 0.02, 'Methods: The mean firing rate was determined by computing the number of spikes for each neuron, in time bins of 1s, then by taking the mean over all neurons. \nThese two measures were then smoothed using a gaussian kernel across time (number of point in window = 100, standard deviation = 50)')

Text(0.12, 0.02, 'Methods: The mean firing rate was determined by computing the number of spikes for each neuron, in time bins of 1s, then by taking the mean over all neurons. \nThese two measures were then smoothed using a gaussian kernel across time (number of point in window = 100, standard deviation = 50)')

## Refaire figure Grosmark2012

In [115]:
rats = ['Rat08', 'Rat09', 'Rat10', 'Rat11']
to_ignore = ['Rat08-20130720', 
             'Rat08-20130722', 
             'Rat08-20130718',
             "Rat09-20140408",
             "Rat09-20140409",
             "Rat10-20140619",
             "Rat10-20140620",
             "Rat10-20140622",
             "Rat10-20140708",
             "Rat11-20150309",
             "Rat11-20150318",
             "Rat11-20150319", 
             "Rat11-20150324",
             "Rat11-20150327",
             "Rat11-20150401"]

syncs = {'syncs_first': [], 'syncs_last': []}

for rat in rats:
    os.chdir(f'E:/DATA/GG-Dataset/{rat}/')
    
    for i in os.listdir(f'E:/DATA/GG-Dataset/{rat}/'):
        if i.endswith('.mat') or i in to_ignore:
            continue

        local_path= f'{rat}/{i}'
        print(local_path)
        bk.load.current_session_linux(base_folder= 'E:/DATA/GG-Dataset/',local_path= local_path)
        states = bk.load.states()
        neurons,metadata = bk.load.spikes() 
        
        print(f"total duration Rem {states['Rem'].tot_length()/1_000_000}")
        
        if states['Rem'].tot_length()/1_000_000 < 100:
            print(f'{local_path} NOT ENOUGH REM')
            continue
            
        neurons_pyr_bla = neurons[(metadata['Type'] == 'Pyr') & (metadata['Region'] == 'BLA')]
        if len(neurons_pyr_bla) < 20:
            print('NOT ENOUGH NEURONS')
            continue
        
        mean_sync_pyr_bla = mean_synchrony(neurons_pyr_bla, window=1, smallbins=0.1, shift=1)
        mean_sync_pyr_bla_s = bk.compute.nts_smooth(mean_sync_pyr_bla, 100,50)

        first, last = get_first_and_last_intervals(states['sws'])

        sync_first = np.nanmean(mean_sync_pyr_bla_s.restrict(first).values)
        sync_last = np.nanmean(mean_sync_pyr_bla_s.restrict(last).values)

        syncs['syncs_first'].append(sync_first)
        syncs['syncs_last'].append(sync_last)
        
syncs_f = np.nanmean(syncs['syncs_first'])
syncs_l = np.nanmean(syncs['syncs_last'])

Rat08/Rat08-20130713
Rat : 8 on day : 6
Working with session Rat08-20130713 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130713
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy
total duration Rem 2371.0


  0%|                                                                                        | 0/22679 [00:00<?, ?it/s]C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
100%|███████████████████████████████████████████████████████████████████████████| 22679/22679 [00:24<00:00, 914.46it/s]


Rat08/Rat08-20130708
Rat : 8 on day : 1
Working with session Rat08-20130708 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130708
Data already saved in Numpy format, loading them from here:
Rat08-20130708-neurons.npy
Rat08-20130708-metadata.npy
total duration Rem 1604.0
NOT ENOUGH NEURONS
Rat08/Rat08-20130709
Rat : 8 on day : 2
Working with session Rat08-20130709 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130709
Data already saved in Numpy format, loading them from here:
Rat08-20130709-neurons.npy
Rat08-20130709-metadata.npy
total duration Rem 1930.0


 21%|███████████████▋                                                           | 5343/25510 [00:03<00:12, 1554.65it/s]C:\Users\maell\AppData\Local\Temp/ipykernel_20972/1913985095.py:31: RuntimeWarning: Mean of empty slice
  mean_sync[i] = np.nanmean(corrcoef) #compute the mean of corrcoef for each window, disregarding the nan
100%|██████████████████████████████████████████████████████████████████████████| 25510/25510 [00:16<00:00, 1527.61it/s]


Rat08/Rat08-20130710
Rat : 8 on day : 3
Working with session Rat08-20130710 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130710
Data already saved in Numpy format, loading them from here:
Rat08-20130710-neurons.npy
Rat08-20130710-metadata.npy
total duration Rem 1480.0


...

Rat11/Rat11-20150330
Rat : 11 on day : 18
Working with session Rat11-20150330 @ E:/DATA/GG-Dataset/Rat11/Rat11-20150330
Data already saved in Numpy format, loading them from here:
Rat11-20150330-neurons.npy
Rat11-20150330-metadata.npy
total duration Rem 1395.0


100%|██████████████████████████████████████████████████████████████████████████| 22385/22385 [00:09<00:00, 2294.78it/s]


Rat11/Rat11-20150331
Rat : 11 on day : 19
Working with session Rat11-20150331 @ E:/DATA/GG-Dataset/Rat11/Rat11-20150331
Data already saved in Numpy format, loading them from here:
Rat11-20150331-neurons.npy
Rat11-20150331-metadata.npy
total duration Rem 1375.0


100%|██████████████████████████████████████████████████████████████████████████| 22776/22776 [00:10<00:00, 2230.77it/s]


Rat11/Rat11-20150402
Rat : 11 on day : 21
Working with session Rat11-20150402 @ E:/DATA/GG-Dataset/Rat11/Rat11-20150402
Data already saved in Numpy format, loading them from here:
Rat11-20150402-neurons.npy
Rat11-20150402-metadata.npy
total duration Rem 528.0
NOT ENOUGH NEURONS
Rat11/Rat11-20150403
Rat : 11 on day : 22
Working with session Rat11-20150403 @ E:/DATA/GG-Dataset/Rat11/Rat11-20150403
Data already saved in Numpy format, loading them from here:
Rat11-20150403-neurons.npy
Rat11-20150403-metadata.npy
total duration Rem 612.0
NOT ENOUGH NEURONS


In [105]:
plt.bar(x = [0,1], height = [syncs_first, syncs_last])

<BarContainer object of 2 artists>

In [109]:
syncs

{'syncs_first': [0.05595993982548164,
  0.09837215103871122,
  0.08954986328116805,
  0.0801525891773762,
  0.07770863417079302,
  0.14954560859850105,
  0.13840551804065906,
  0.13808564506020177,
  0.16892326380121997,
  0.13805186539422795,
  0.062272548017484254,
  0.046435920751681584,
  0.058056353313524496,
  0.09631179770742211,
  0.05395753109192529,
  0.07638939733389433,
  0.09031993883250389,
  0.1100050755089742,
  0.15445707962390867,
  0.156397740535071,
  0.12432405523303211,
  0.11436723976166815,
  0.0485585312838301,
  0.046929096822757616,
  0.08222780643940973,
  0.1801191025030225,
  0.14431532298716576],
 'syncs_last': [0.054664034201258434,
  0.09954331558783497,
  0.07557692703469587,
  0.06833118326793622,
  0.0674458899698727,
  0.12290378342373368,
  0.14571669211292493,
  0.13683434923876284,
  0.15333171704690574,
  0.13148693767598382,
  0.05770680875221293,
  0.05282859932611181,
  0.06622672764399425,
  0.07683442171442934,
  0.0647004317075495,
  0.080

In [116]:
syncs = pd.DataFrame(syncs)
sns.boxplot(data = syncs, whis=np.inf)
sns.swarmplot(data = syncs, color=".2")
plt.ylim(0.025, 0.225)

(0.025, 0.225)

In [113]:
sp.stats.kruskal(syncs['syncs_first'], syncs['syncs_last'], nan_policy='omit')

KruskalResult(statistic=0.0018705574261161928, pvalue=0.9655022948721989)

# Refaire fig Grosmark2012 pour extended sleep

In [183]:
rats = ['Rat08', 'Rat09', 'Rat10', 'Rat11']
to_ignore = ['Rat08-20130720', 
             'Rat08-20130722', 
             'Rat08-20130718',
             "Rat09-20140408",
             "Rat09-20140409",
             "Rat10-20140619",
             "Rat10-20140620",
             "Rat10-20140622",
             "Rat10-20140708",
             "Rat11-20150309",
             "Rat11-20150318",
             "Rat11-20150319", 
             "Rat11-20150324",
             "Rat11-20150327",
             "Rat11-20150401"]

syncs = {'syncs_first': [], 'syncs_last': []}

nb_ext_sleep = 0 

for rat in rats:
    os.chdir(f'E:/DATA/GG-Dataset/{rat}/')
    
    for i in os.listdir(f'E:/DATA/GG-Dataset/{rat}/'):
        if i.endswith('.mat') or i in to_ignore:
            continue

        local_path= f'{rat}/{i}'
        print(local_path)
        bk.load.current_session_linux(base_folder= 'E:/DATA/GG-Dataset/',local_path= local_path)
        states = bk.load.states()
        neurons,metadata = bk.load.spikes() 
        
        print(f"total duration Rem {states['Rem'].tot_length()/1_000_000}")
        
        if states['Rem'].tot_length()/1_000_000 < 100:
            print(f'{local_path} NOT ENOUGH REM')
            continue
            
        neurons_pyr_bla = neurons[(metadata['Type'] == 'Pyr') & (metadata['Region'] == 'BLA')]
        if len(neurons_pyr_bla) < 20:
            print('NOT ENOUGH NEURONS')
            continue
        
        mean_sync_pyr_bla = mean_synchrony(neurons_pyr_bla, window=1, smallbins=0.1, shift=1)
        mean_sync_pyr_bla_s = bk.compute.nts_smooth(mean_sync_pyr_bla, 100,50)
        
        ext_sleeps = get_extended_sleep(states, 30*60, 60)
        for ext_sleep_start, ext_sleep_end in zip(ext_sleeps['start'],ext_sleeps['end']):
            
            nb_ext_sleep += 1
            
            ext_sleep = nts.IntervalSet(ext_sleep_start, ext_sleep_end)
            
            first, last = get_first_and_last_intervals(states['sws'].intersect(ext_sleep))

            sync_first = np.nanmean(mean_sync_pyr_bla_s.restrict(first).values)
            sync_last = np.nanmean(mean_sync_pyr_bla_s.restrict(last).values)

            syncs['syncs_first'].append(sync_first)
            syncs['syncs_last'].append(sync_last)
        
syncs_f = np.nanmean(syncs['syncs_first'])
syncs_l = np.nanmean(syncs['syncs_last'])

Rat08/Rat08-20130713
Rat : 8 on day : 6
Working with session Rat08-20130713 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130713
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy
total duration Rem 2371.0


  0%|                                                                                        | 0/22679 [00:00<?, ?it/s]C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\maell\anaconda3\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
100%|██████████████████████████████████████████████████████████████████████████| 22679/22679 [00:12<00:00, 1806.37it/s]


Rat08/Rat08-20130708
Rat : 8 on day : 1
Working with session Rat08-20130708 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130708
Data already saved in Numpy format, loading them from here:
Rat08-20130708-neurons.npy
Rat08-20130708-metadata.npy
total duration Rem 1604.0
NOT ENOUGH NEURONS
Rat08/Rat08-20130709
Rat : 8 on day : 2
Working with session Rat08-20130709 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130709
Data already saved in Numpy format, loading them from here:
Rat08-20130709-neurons.npy
Rat08-20130709-metadata.npy
total duration Rem 1930.0


 20%|███████████████▎                                                           | 5229/25510 [00:01<00:05, 3636.40it/s]C:\Users\maell\AppData\Local\Temp/ipykernel_20972/1913985095.py:31: RuntimeWarning: Mean of empty slice
  mean_sync[i] = np.nanmean(corrcoef) #compute the mean of corrcoef for each window, disregarding the nan
100%|██████████████████████████████████████████████████████████████████████████| 25510/25510 [00:07<00:00, 3620.35it/s]


Rat08/Rat08-20130710
Rat : 8 on day : 3
Working with session Rat08-20130710 @ E:/DATA/GG-Dataset/Rat08/Rat08-20130710
Data already saved in Numpy format, loading them from here:
Rat08-20130710-neurons.npy
Rat08-20130710-metadata.npy
total duration Rem 1480.0


...

Rat11/Rat11-20150330
Rat : 11 on day : 18
Working with session Rat11-20150330 @ E:/DATA/GG-Dataset/Rat11/Rat11-20150330
Data already saved in Numpy format, loading them from here:
Rat11-20150330-neurons.npy
Rat11-20150330-metadata.npy
total duration Rem 1395.0


100%|██████████████████████████████████████████████████████████████████████████| 22385/22385 [00:04<00:00, 4647.72it/s]


Rat11/Rat11-20150331
Rat : 11 on day : 19
Working with session Rat11-20150331 @ E:/DATA/GG-Dataset/Rat11/Rat11-20150331
Data already saved in Numpy format, loading them from here:
Rat11-20150331-neurons.npy
Rat11-20150331-metadata.npy
total duration Rem 1375.0


100%|██████████████████████████████████████████████████████████████████████████| 22776/22776 [00:04<00:00, 5311.88it/s]


Rat11/Rat11-20150402
Rat : 11 on day : 21
Working with session Rat11-20150402 @ E:/DATA/GG-Dataset/Rat11/Rat11-20150402
Data already saved in Numpy format, loading them from here:
Rat11-20150402-neurons.npy
Rat11-20150402-metadata.npy
total duration Rem 528.0
NOT ENOUGH NEURONS
Rat11/Rat11-20150403
Rat : 11 on day : 22
Working with session Rat11-20150403 @ E:/DATA/GG-Dataset/Rat11/Rat11-20150403
Data already saved in Numpy format, loading them from here:
Rat11-20150403-neurons.npy
Rat11-20150403-metadata.npy
total duration Rem 612.0
NOT ENOUGH NEURONS


In [180]:
nb_ext_sleep

23

In [184]:
syncs = pd.DataFrame(syncs)
sns.boxplot(data = syncs, whis=np.inf)
sns.swarmplot(data = syncs, color=".2")
plt.ylim(0.025, 0.225)

(0.025, 0.225)

In [182]:
sp.stats.kruskal(syncs['syncs_first'], syncs['syncs_last'], nan_policy='omit')

KruskalResult(statistic=0.28473262545219685, pvalue=0.5936159314613576)

# Trash/Tests

In [64]:
mean_sync_pyr_bla_s.restrict(last).values

array([0.09744055, 0.09773763, 0.09958807, 0.10034445, 0.09996849,
       0.10018768, 0.10086398, 0.10125544, 0.10142914, 0.10303499,
       0.103777  , 0.10479481, 0.10650194, 0.10701476, 0.10755754,
       0.10782182, 0.10755634, 0.10771072, 0.10761934, 0.10756761,
       0.10772372, 0.10912454, 0.11026872, 0.110522  , 0.11083997,
       0.11047307, 0.11030074, 0.11022431, 0.11016118, 0.11091922,
       0.11166068, 0.11190468, 0.11224382, 0.11193829, 0.11233547,
       0.11154848, 0.11126355, 0.11020312, 0.11140478, 0.11152033,
       0.11135788, 0.11138613, 0.11041421, 0.1072486 , 0.10791942,
       0.10805303, 0.10795152, 0.10865544, 0.10864916, 0.10862195,
       0.10861453, 0.10863849, 0.11016284, 0.11132082, 0.11193651,
       0.11237149, 0.112424  , 0.1124879 , 0.11452956, 0.11472824,
       0.11501851, 0.11554209, 0.11500701, 0.11485533, 0.11565237,
       0.11573111, 0.11589273, 0.11604934, 0.1153057 , 0.11539583,
       0.11563886, 0.11620121, 0.11544586, 0.11580392, 0.11640

In [60]:
states['Rem'].iloc[-1]
last = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in states['Rem'].iloc[-1].items() ]))
last = nts.IntervalSet(last)

In [130]:
np.sum(np.isnan(mean_sync_pyr_bla_s.values))

0

In [91]:
test = {'Rem BLA': [], 'sws BLA': [], 'Rem Hpc' : [], 'sws Hpc': []}
for i, j  in zip(states['Rem'].as_units('s').start, states['Rem'].as_units('s').end):
    interval = nts.IntervalSet(i,j, time_units='s')
    test['Rem BLA'].append(np.mean(mean_sync_pyr_bla.restrict(interval).values))
    test['Rem Hpc'].append(np.mean(mean_sync_pyr_hpc.restrict(interval).values))
for i, j  in zip(states['sws'].as_units('s').start, states['sws'].as_units('s').end):
    interval = nts.IntervalSet(i,j, time_units='s')
    test['sws BLA'].append(np.mean(mean_sync_pyr_bla.restrict(interval).values))
    test['sws Hpc'].append(np.mean(mean_sync_pyr_hpc.restrict(interval).values))